In [ ]:
query = {
    "min_time": "2023-06-01T00:00:00",
    "max_time": "2023-11-30T23:59:59",
    "min_lat": 10,
    "max_lat": 42,
    "min_lon": -99,
    "max_lon": -50,
}

In [ ]:
from ioos_metrics.ioos_metrics import _ngdac_gliders as ngdac_gliders


import pandas as pd

metadata_metrics = ngdac_gliders(**query)
metadata_metrics["time_coverage_start"] = pd.to_datetime(
    metadata_metrics["time_coverage_start"],
)
metadata_metrics["time_coverage_end"] = pd.to_datetime(
    metadata_metrics["time_coverage_end"],
)

_save_df = (
    metadata_metrics[
        [
            "glider",
            "wmo_id",
            "sea_name",
            "time_coverage_start",
            "time_coverage_end",
            "days",
            "num_profiles",
            "institution",
            "acknowledgment",
        ]
    ]
    .reset_index(
        names=["dataset_id"],
    )
    .sort_values(
        by="dataset_id",
        key=lambda x: x.str.lower(),
    )
    .rename(
        columns={
            "time_coverage_start": "start_date",
            "time_coverage_end": "end_date",
            "sea_name": "deployment_area",
        },
    )
)

_save_df["start_date"] = _save_df["start_date"].dt.strftime("%m-%d-%Y")
_save_df["end_date"] = _save_df["end_date"].dt.strftime("%m-%d-%Y")
_save_df["days"] = [d.days for d in _save_df["days"]]

_save_df.to_csv(
    path_or_buf="glider_metadata_metrics.csv",
    index=False,
)

In [ ]:
print(f'Glider days: {metadata_metrics["days"].sum().days}')
print(f'Glider deployments: {len(metadata_metrics)}')
print(f'Number of profiles days: {metadata_metrics["num_profiles"].sum()}')